<a href="https://colab.research.google.com/github/Ananya-AJ/Deep-Learning/blob/main/Assignment4/k_data_augmentation_classification__video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Note: restart runtime after this import before running the augmentations
!pip install -U augly[video]
!sudo apt-get install python3-magic

In [ ]:
!wget -c http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
!unrar x hmdb51_org.rar

In [ ]:
!unrar x shoot_gun.rar

In [ ]:
!unrar x brush_hair.rar

In [4]:
!pip install opencv-python-headless
!pip install av

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 40.9 MB/s eta 0:00:00


In [5]:
import cv2
import numpy as np
import os

data_dir_1 = './brush_hair'
data_dir_2 = './shoot_gun'
img_size = 64

# Read frames from videos in the first directory
data_1 = []
for i, filename in enumerate(os.listdir(data_dir_1)):
    if i == 5:
        break
    video_path = os.path.join(data_dir_1, filename)
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_size, img_size))
        data_1.append(frame)
    cap.release()

# Read frames from videos in the second directory
data_2 = []
for i, filename in enumerate(os.listdir(data_dir_2)):
    if i == 5:
        break
    video_path = os.path.join(data_dir_2, filename)
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_size, img_size))
        data_2.append(frame)
    cap.release()

# Create labels for the data
labels_1 = np.zeros(len(data_1))
labels_2 = np.ones(len(data_2))

# Combine the data and labels
data = np.array(data_1 + data_2)
labels = np.concatenate((labels_1, labels_2))

# Shuffle the data and labels
idx = np.random.permutation(len(data))
data = data[idx]
labels = labels[idx]

# Split the data into training and testing sets
split = int(0.8 * len(data))
x_train = data[:split]
y_train = labels[:split]
x_test = data[split:]
y_test = labels[split:]


In [6]:
import tensorflow as tf

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
39/39 [==============================] - 12s 21ms/step - loss: 2.5785 - accuracy: 0.9123 - val_loss: 3.3419e-05 - val_accuracy: 1.0000
Epoch 2/10
39/39 [==============================] - 0s 8ms/step - loss: 1.3728e-05 - accuracy: 1.0000 - val_loss: 1.1320e-06 - val_accuracy: 1.0000
Epoch 3/10
39/39 [==============================] - 0s 8ms/step - loss: 4.6332e-07 - accuracy: 1.0000 - val_loss: 4.2393e-07 - val_accuracy: 1.0000
Epoch 4/10
39/39 [==============================] - 0s 7ms/step - loss: 2.2619e-07 - accuracy: 1.0000 - val_loss: 2.6564e-07 - val_accuracy: 1.0000
Epoch 5/10
39/39 [==============================] - 0s 9ms/step - loss: 1.6372e-07 - accuracy: 1.0000 - val_loss: 2.0874e-07 - val_accuracy: 1.0000
Epoch 6/10
39/39 [==============================] - 0s 8ms/step - loss: 1.3379e-07 - accuracy: 1.0000 - val_loss: 1.8221e-07 - val_accuracy: 1.0000
Epoch 7/10
39/39 [==============================] - 0s 9ms/step - loss: 1.1651e-07 - accuracy: 1.0000 - val_loss: 

Accuracy is low as we are only using 5 videos of each class to train, this experiment is just to check if anything changes from pre data augmentation to post data augmentation. The colab crashes with more number of videos

Data Augmentation using augly video, adding noise and blurring the videos

In [66]:
import cv2
import numpy as np
import os
import augly.video as vidaugs

# Define the augmentations to be applied to the videos
aug = vidaugs.Compose([
    vidaugs.AddNoise(),
    vidaugs.Blur(sigma=5.0),
    vidaugs.OverlayDots(),
])

# Define the paths to the video files
videos_path = "./"
shoot_gun_path = os.path.join(videos_path, "shoot_gun")
brush_hair_path = os.path.join(videos_path, "brush_hair")

# Define the size of the video frames
img_size = 224

# Define the data lists
data_1 = []
labels_1 = []
data_2 = []
labels_2 = []

# Read the first 5 videos from the shoot_gun folder
for i, filename in enumerate(os.listdir(shoot_gun_path)[:5]):
    if filename.endswith(".avi"):
        filepath = os.path.join(shoot_gun_path, filename)
        cap = cv2.VideoCapture(filepath)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (img_size, img_size))
            writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (img_size, img_size), True)
            writer.write(frame)
            writer.release()
            aug_video = cv2.VideoCapture("output.avi")
            aug_frames = []
            while aug_video.isOpened():
                ret, frame = aug_video.read()
                if not ret:
                    break
                aug_frames.append(frame)
            aug_video.release()
            os.remove("output.avi")
            aug_data = np.array(aug_frames)
            data_1.append(aug_data)
        cap.release()

# Read the first 5 videos from the brush_hair folder
for i, filename in enumerate(os.listdir(brush_hair_path)[:5]):
    if filename.endswith(".avi"):
        filepath = os.path.join(brush_hair_path, filename)
        cap = cv2.VideoCapture(filepath)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (img_size, img_size))
            writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (img_size, img_size), True)
            writer.write(frame)
            writer.release()
            aug_video = cv2.VideoCapture("output.avi")
            aug_frames = []
            while aug_video.isOpened():
                ret, frame = aug_video.read()
                if not ret:
                    break
                aug_frames.append(frame)
            aug_video.release()
            os.remove("output.avi")
            aug_data = np.array(aug_frames)
            data_2.append(aug_data)
        cap.release()

print(len(data_1))
print(len(data_2))

# Convert the data lists to numpy arrays
data_1 = np.concatenate(data_1, axis=0)
data_2 = np.concatenate(data_2, axis=0)


# Create the labels for the data
labels_1 = np.zeros((data_1.shape[0],))
labels_2 = np.ones((data_2.shape[0],))

# Merge the data and labels
X = np.concatenate((data_1, data_2))
y = np.concatenate((labels_1, labels_2))

# Shuffle the data and labels
p = np.random.permutation(X.shape[0])
X = X[p]
y = y[p]

# Split the data and labels into training and testing sets
split = int(0.8 * X.shape[0])
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


463
1091


In [67]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1243, 224, 224, 3)
(311, 224, 224, 3)
(1243,)
(311,)


In [55]:

# img_size = 224

# # Reshape the data to be used in the 3D CNN
# X_train = X_train.reshape((-1, 16, img_size, img_size, 3))
# y_train = y_train.reshape((-1, 16, img_size, img_size, 3))


In [68]:
import tensorflow as tf
from tensorflow import keras

# Define the CNN model architecture
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
39/39 [==============================] - 7s 83ms/step - loss: 16.3066 - accuracy: 0.9276 - val_loss: 3.7606e-04 - val_accuracy: 1.0000
Epoch 2/10
39/39 [==============================] - 2s 48ms/step - loss: 4.4585e-04 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 0.9968
Epoch 3/10
39/39 [==============================] - 2s 48ms/step - loss: 3.3629e-04 - accuracy: 1.0000 - val_loss: 4.6631e-06 - val_accuracy: 1.0000
Epoch 4/10
39/39 [==============================] - 2s 49ms/step - loss: 8.6647e-05 - accuracy: 1.0000 - val_loss: 1.0848e-06 - val_accuracy: 1.0000
Epoch 5/10
39/39 [==============================] - 2s 49ms/step - loss: 2.3000e-06 - accuracy: 1.0000 - val_loss: 3.2160e-06 - val_accuracy: 1.0000
Epoch 6/10
39/39 [==============================] - 2s 48ms/step - loss: 2.8489e-05 - accuracy: 1.0000 - val_loss: 3.9091e-07 - val_accuracy: 1.0000
Epoch 7/10
39/39 [==============================] - 2s 48ms/step - loss: 1.0323e-06 - accuracy: 1.0000 - val_loss

In [69]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

10/10 [==============================] - 0s 17ms/step - loss: 1.5510e-08 - accuracy: 1.0000
Test accuracy: 1.0


val accuracy doesnt change as we have only 5 videos for each class , however the loss reduces considerabley from 8.7 to 4.2. More than 5 videos, colab crashes